In [5]:
import numpy as np
import pandas as pd
import datetime
import math
import random
from datetime import datetime
from statistics import mean
from sklearn import linear_model

In [6]:
x = [
    [1,2],
    [1,3],
    [2,3],
    [2,4],
    [3,2],
    [3,5],
    [4,1],
    #A partir de aqui es relleno para el mini-batch
    [1,2],
    [1,3],
    [2,3],
    [2,4],
    [3,2],
    [3,5],
    [4,1]
]
y = [1.03,-1.44,4.53,2.24,13.27,5.62,21.53,#A partir de aqui es relleno para el mini-batch
     1.03,-1.44,4.53,2.24,13.27,5.62,21.53]

In [7]:
def signo(x):
  if x>=0:
    return 1
  else:
    return -1

In [8]:
def predict(modelo,x):
  if type(x[0]) == list:
    pred_v = []
    for x_it in x:
      pred = modelo[len(modelo)-1]
      for i in range(len(modelo)-1):
        pred+= modelo[i]*x_it[i]
      pred_v.append(pred)
    return pred_v
  else:
    pred = modelo[len(modelo)-1]
    for i in range(len(modelo)-1):
      pred+= modelo[i]*x[i]

    return pred

def MAE(modelo,x,y):
  pred = predict(modelo,x)
  errs =[e1 - e2 for e1, e2 in zip(y,pred)]
  abs_v = [abs(ele) for ele in errs]

  return mean(abs_v)
def MSE(modelo,x,y):
  pred = predict(modelo,x)
  errs =[e1 - e2 for e1, e2 in zip(y,pred)]
  squ_v = [(ele)**2 for ele in errs]

  return mean(squ_v)

def MSE_GRAD(modelo,x,errs,w_it,l_rate):
  if w_it < len(modelo)-1:
        #len(modelo)-1 porque el último peso es el sesgo y se ajusta
        #de forma distinta
        #Aqui se ajustan todos los pesos menos el bias

        #Cálculo del Σ((Ŷ-Y)*Xji)
        for er in range(len(errs)):
          errs[er]= errs[er]*x[er][w_it]
        modelo[w_it] = modelo[w_it]-1*(2*l_rate)*mean(errs)

  else:
    #Aqui se ajusta el bias
    modelo[w_it] = modelo[w_it] -1*(2*l_rate*mean(errs) )

def MAE_GRAD(modelo,x,errs,w_it,l_rate):
  if w_it < len(modelo)-1:
        for er in range(len(errs)):
          errs[er]= signo(errs[er])*x[er][w_it]
        modelo[w_it] = modelo[w_it]-1*(l_rate)*mean(errs)
        #Aqui se ajusta el bias
  else:
    errs = [signo(x) for x in errs]
    modelo[w_it] = modelo[w_it] -1*(l_rate*mean(errs) )

In [9]:
predict([1,1,1],x)

[4, 5, 6, 7, 6, 9, 6, 4, 5, 6, 7, 6, 9, 6]

In [10]:
modelo = [0,0,0]
#[x1,x2,b]

In [11]:
def generate_mini_batch(x, y, batch_percentage):
  tam_mini_batch = math.trunc(len(x)*batch_percentage)
  mini_batch_x = []
  mini_batch_y = []
  x_copy = x.copy()
  y_copy = y.copy()

  while len(x_copy)>0:
    it_mini_batchx = []
    it_mini_batchy = []
    while len(it_mini_batchx)<tam_mini_batch:
      index = random.randint(0,len(x_copy)-1)
      it_mini_batchx.append(x_copy[index])
      it_mini_batchy.append(y_copy[index])
      x_copy.pop(index)
      y_copy.pop(index)
    mini_batch_x.append(it_mini_batchx)
    mini_batch_y.append(it_mini_batchy)

  return mini_batch_x,mini_batch_y


In [12]:
def linear_regression(modelo,x,y,n_it,l_rate,func_ajuste, minibatch_mode, batch_percentage=None):
  for _ in range(n_it):
    if not minibatch_mode:
      errs = []
      pred = predict(modelo,x)
      errs = [e1 - e2 for e1, e2 in zip(pred,y)]

      for w_it in range(len(modelo)):
        errs_it = errs.copy()
        func_ajuste(modelo,x,errs_it,w_it,l_rate)
    else:
      batch_x, batch_y = generate_mini_batch(x, y, batch_percentage)
      for a, b in zip(batch_x, batch_y):
        errs = []
        pred = predict(modelo,a)
        errs = [e1 - e2 for e1, e2 in zip(pred,b)]

        for w_it in range(len(modelo)):
          errs_it = errs.copy()
          func_ajuste(modelo,a,errs_it,w_it,l_rate)
  return modelo

In [13]:
# Ajuste usando batch

print("Ajuste usando batch")

modelo = [1,1,1]
print('Ajuste con MSE')
print('Error inicial -> ',MSE(modelo,x,y))
linear_regression(modelo,x,y,20000,0.001,MSE_GRAD, False)
print('Error final -> ',MSE(modelo,x,y))
print(modelo)

modelo = [1,1,1]
print('\nAjuste con MAE')
print('Error inicial -> ',MAE(modelo,x,y))
linear_regression(modelo,x,y,20000,0.001,MAE_GRAD, False)
print('Error final -> ',MAE(modelo,x,y))
print(modelo)

# Ajuste usando minibatch

print("\nAjuste usando minibatch")
modelo = [1,1,1]
print('Ajuste con MSE')
print('Error inicial -> ',MSE(modelo,x,y))
linear_regression(modelo,x,y,20000,0.001,MSE_GRAD, True, 0.2)
print('Error final -> ',MSE(modelo,x,y))
print(modelo)

modelo = [1,1,1]
print('\nAjuste con MAE')
print('Error inicial -> ',MAE(modelo,x,y))
linear_regression(modelo,x,y,20000,0.001,MAE_GRAD, True, 0.2)
print('Error final -> ',MAE(modelo,x,y))
print(modelo)

Ajuste usando batch
Ajuste con MSE
Error inicial ->  54.367314285714286
Error final ->  0.008417023656379178
[6.007147684087246, -2.489069952946704, 0.06976974256473045]

Ajuste con MAE
Error inicial ->  5.974285714285714
Error final ->  0.05959183673453323
[6.008285714285928, -2.4745714285724416, -0.02685714285709419]

Ajuste usando minibatch
Ajuste con MSE
Error inicial ->  54.367314285714286
Error final ->  0.007877000423875066
[6.024202916706965, -2.474892028683118, -0.015306286927502773]

Ajuste con MAE
Error inicial ->  5.974285714285714
Error final ->  0.060285714285817936
[6.010000000000454, -2.474999999999909, -0.0320000000000009]


Transformación del dataset a arrays para que el regresor pueda trabajar con él

In [14]:
data = pd.read_csv('diabetes.tab.csv',delimiter='\t')
def from_dataset_to_arrays(data):
  Y = data['Y']
  y = list(Y.values)
  data = data.drop(columns='Y')
  x = list(data.values)
  for a in range(len(x)):
    b = list(x[a])
    x[a] = b

  return x,y
x,y = from_dataset_to_arrays(data)

FileNotFoundError: [Errno 2] No such file or directory: 'diabetes.tab.csv'

Nota, aquí si no se utiliza un learning rate pequeño no minimiza bien el error

In [ ]:
modelo = []
for i in range(len(x[0])):
  modelo.append(1)
print('Ajuste con MSE')
print('Error inicial -> ',MAE(modelo,x,y))
modelo =linear_regression(modelo,x,y,1000,0.00001,MSE_GRAD,False)
print('Error final -> ',MAE(modelo,x,y))
print(modelo)

Ajuste con MSE
Error inicial ->  382.9552796380091
Error final ->  46.55631306348672
[-0.09301242361673175, -0.2118347205229997, 5.170857536195489, 0.726097690015385, 1.3607173975557387, -1.5565579473985036, -2.681144560201426, 0.6958787173297917, 1.0526176919558023, 0.8364888144335826]


In [15]:
reg = linear_model.LinearRegression()
reg.fit(x, y)

print(f'[{reg.coef_} {reg.intercept_}]')


[[ 6.02426471 -2.47485294] -0.015882352941170907]
